# Fix HTTP 403 Error When Downloading Images

This notebook demonstrates how to download images using Python requests while avoiding the **HTTP 403 Forbidden** error.

## The Problem
When downloading images from certain URLs, you may encounter:
```
HTTPError: 403 Client Error: Forbidden
```

## The Solution
Add proper HTTP headers (especially `User-Agent`) to make the request appear as if it's coming from a legitimate browser.

In [ ]:
# Install required packages if needed
# !pip install requests pillow matplotlib

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

## Method 1: Basic Solution with User-Agent Header

In [ ]:
# The image URL
url = "http://bit.ly/46xv3sL"

# Add User-Agent header to avoid 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Download the image
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an error for bad status codes

# Open and display the image
img = Image.open(BytesIO(response.content))

plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.axis('off')
plt.title('Image Downloaded Successfully')
plt.show()

print(f"Image size: {img.size}")
print(f"Image mode: {img.mode}")

## Method 2: Complete Solution with Full Browser Headers

In [ ]:
def download_image(url, save_path=None):
    """
    Download an image with proper headers to avoid 403 errors.
    
    Args:
        url (str): Image URL
        save_path (str, optional): Path to save the image locally
    
    Returns:
        PIL.Image: The downloaded image
    """
    # Full set of headers to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    try:
        # Download with headers
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Open image
        img = Image.open(BytesIO(response.content))
        
        # Save if path provided
        if save_path:
            img.save(save_path)
            print(f"✓ Image saved to: {save_path}")
        
        return img
    
    except requests.exceptions.HTTPError as e:
        print(f"✗ HTTP Error: {e}")
        raise
    except Exception as e:
        print(f"✗ Error: {e}")
        raise

In [ ]:
# Download and display the image
url = "http://bit.ly/46xv3sL"
img = download_image(url, save_path="downloaded_image.png")

# Display
plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.axis('off')
plt.title('Downloaded Image')
plt.show()

## Method 3: Using Requests Session for Multiple Downloads

In [ ]:
# Create a session with persistent headers
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
})

# Now you can download multiple images without repeating headers
response = session.get(url)
response.raise_for_status()

img = Image.open(BytesIO(response.content))
plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.axis('off')
plt.show()

## Explanation

### Why does this work?

1. **User-Agent Header**: Many servers block requests that don't have a User-Agent header or have suspicious ones (like the default Python requests User-Agent)

2. **Browser Mimicry**: By adding headers that mimic a real browser, the server treats our request as legitimate

3. **Common Headers**:
   - `User-Agent`: Identifies the client (browser)
   - `Accept`: Specifies what content types the client can handle
   - `Accept-Language`: Preferred languages
   - `Accept-Encoding`: Supported compression methods

### Key Points:
- The minimal fix is just adding a `User-Agent` header
- For maximum compatibility, use a full set of browser headers
- Use a session object if downloading multiple images
- Always handle exceptions properly